# Como treinar YOLOv3 usando Darknet em um notebook Colab e otimizar o tempo de carregamento da VM

## Bem-vindo!

Este notebook Colab mostrará como:

* Treinar um modelo **Yolo v3** usando **Darknet** usando a GPU Colab de **12 GB de RAM**.
* Transformar os notebooks Colab em uma ferramenta eficaz para trabalhar em projetos reais. Lidando com a desvantagem de um tempo de execução que **explode a cada 12 horas**!
* Trabalhando diretamente dos arquivos no seu computador.
* Configure seu notebook para instalar tudo o que você precisa e começar a treinar em cerca de um minuto (testado com um conjunto de dados de 550 MB).
* Receba seus pesos treinados diretamente no seu computador durante o treinamento. Enquanto o notebook treina, você pode verificar como ele está usando os pesos treinados no seu computador.

#### Este notebook faz parte da publicação [Como treinar o YOLOv3 usando o framework Darknet e otimizar os tempos de carregamento da VM em tempo de execução](http://blog.ibanyez.info/blogs/coding/20190410-run-a-google-colab-notebook-to-train-yolov3-using-darknet-in/). Recomendo sua visita! Você encontrará uma explicação mais aprofundada sobre o Google Colab, os benefícios e as limitações desta excelente ferramenta.

### Estes são os passos que seguiremos:

* Configurar o Google Drive e mapeá-lo como _Drive_ de rede.
* Alguns utilitários para auxiliar em algumas tarefas.
* Configurar os pré-requisitos no tempo de execução.
* Verificar a instalação do CUDA na VM em tempo de execução.
* Instalar o cuDNN.
* Clonar e compilar o Darknet. Usaremos um repositório baseado no [repositório Darknet do AlexeyAB](https://github.com/AlexeyAB/darknet/). Apliquei algumas alterações para possibilitar o carregamento de arquivos de `/Meu Drive/` e reduzi o número de logs no console para acelerar o notebook.
* Verificaremos se tudo funciona perfeitamente.
* Explicamos como gerenciar seus arquivos YOLO no seu computador e eles serão usados ​​de forma transparente a partir deste notebook.

> _**NOTA:** Células com a anotação **`# Célula não necessária`**. Podem ser removidas sem qualquer impacto. São apenas conteúdo explicativo._

### Sem mais delongas, vamos começar!


## PASSO 0. Configurar o tempo de execução para funcionar com a GPU



Queremos usar a aceleração de hardware da GPU com **12 GB de RAM**!

Acesse **> Menu > Tempo de Execução > Configurar Tipo de Tempo de Execução** e selecione **GPU** no menu suspenso **Acelerador de Hardware**

## PASSO 1. Conecte seus arquivos ao Google Drive


Para ter seus arquivos no seu computador local, você precisa instalar e configurar o Backup e Sincronização do Google para manter uma pasta do seu Drive sincronizada com uma pasta no seu computador.



1. Crie uma pasta no seu Google Drive chamada _**darknet**_
2. Configure o Backup e Sincronização do Google da seguinte forma. Se você não fala espanhol, talvez aprenda algumas palavras! **;)**


**Após esta etapa, você terá uma pasta chamada _darknet_ no seu computador local. Esta pasta é onde você trabalhará com os arquivos do seu computador**

> _**DICA** - Precisamos ter um bom desempenho ao baixar dados do Drive para o Colab. Ter muitos arquivos na pasta raiz do Drive pode deixar tudo bastante lento. É uma boa prática trabalhar com o Colab para mover todos os arquivos da pasta raiz para uma pasta_

## PASSO 2. Conecte o notebook Colab ao Google Drive



Agora, vamos mapear sua pasta do Google Drive. Este primeiro passo é o único que exigirá sua interação manual sempre que você executar o notebook.

* Execute a seguinte célula _(Clique no botão Reproduzir ou pressione CTRL + ENTER)_ e clique no link para autorizar seu notebook a acessar o Google Drive.
* Cole o código que o Google fornecerá a você e pressione `Enter`.

In [ ]:
# Esta célula importa a biblioteca do Drive e monta seu Google Drive como uma unidade local de VM. Você pode acessar seus arquivos do Drive
# usando este caminho "/content/gdrive/MyDrive/"

from google.colab import drive
# drive.mount('/content/gdrive')
drive.mount('/content/drive')

Parabéns! Agora você pode acessar a pasta local do seu computador diretamente daqui!

Confira!

In [ ]:
# Célula não necessária
# Liste o conteúdo da pasta local do seu computador
# !ls -la "/content/gdrive/My Drive/darknet"
!ls -la "/content/drive/My Drive/darknet"

In [ ]:
!sudo apt-get install tree

In [ ]:
# !tree /content/gdrive/My\ Drive/darknet/
!tree /content/drive/My\ Drive/darknet/

### NOTA DE ATUALIZAÇÃO



Se quiser simplificar seus caminhos, você pode usar um link simbólico:

`!ln -s "/content/gdrive/My Drive/darknet/" /mydrive`

Assim, você poderá acessar seus arquivos do Google Drive usando apenas o caminho `/mydrive`

**CUIDADO: Todos os caminhos neste bloco de notas não usam links simbólicos. Lembre-se de alterar o caminho em todos os lugares**

Agradecemos a **Dennis Kashkin** pela recomendação!

In [ ]:
# Uncomment if you want to use Symbolic link
#!ln -s /content/gdrive/My\ Drive/darknet/ /mydrive
#!ls /mydrive

## PASSO 2. Verifique a versão de lançamento do CUDA



O Nvidia CUDA vem pré-instalado nos notebooks Colab. Agora, vamos verificar a versão instalada.

> _**ATENÇÃO:** Talvez em algum momento, a partir do momento em que escrevo estas linhas (19 de setembro de 2025), a versão do CUDA seja atualizada no Colab e você deva baixar outra versão do cuDNN na próxima etapa. Agora é a versão 12.5 e estamos usando o cuDNN (cudnn-10.0-linux-x64-v7.5.0.56.tgz) conforme necessário_

In [ ]:
# This cell can be commented once you checked the current CUDA version
# CUDA: Let's check that Nvidia CUDA is already pre-installed and which version is it. In some time from now maybe you
!/usr/local/cuda/bin/nvcc --version


In [ ]:
# !sudo apt-get -y install cudnn9-cuda-12

In [ ]:
# !sudo apt-get -y install cudnn9-cuda-13

In [ ]:
# 1) Informação da distribuição (preferível)
!cat /etc/os-release

# 2) Informação padronizada de distribuição (se disponível)
!lsb_release -a

# 3) Informação do kernel e arquitetura
!uname -a

## PASSO 3. Instale o cuDNN de acordo com a versão atual do CUDA


Agora, você precisa baixar o cuDNN do site da Nvidia. Você precisará se cadastrar no site.

* Baixe o cuDNN do [site da Nvidia](https://developer.nvidia.com/cudnn)

* No momento, como temos o _**CUDA 10.0**_ pré-instalado no ambiente de execução do Colab, você precisa baixar o [cuDNN v7.5.0.56 para CUDA v10.0](https://developer.nvidia.com/compute/machine-learning/cudnn/secure/v7.5.0.56/prod/10.0_20190219/cudnn-10.0-linux-x64-v7.5.0.56.tgz) - o arquivo é cudnn-10.0-linux-x64-v7.5.0.56.tgz

* No seu computador local, crie uma pasta chamada _**cuDNN**_ na sua pasta local _**darknet**_. Copie o arquivo _**tgz**_ para lá



In [ ]:
## Crie pasta em darknet e mova o .deb:
# !cd "/content/gdrive/MyDrive/darknet/"
# !cd /content/gdrive/MyDrive/darknet
%cd /content/drive/MyDrive/darknet
# !mkdir -p cuDNN

In [ ]:
ls

In [ ]:
# mv ~/Downloads/cudnn-local-repo-ubuntu2204-9.13.0_1.0-1_amd64.deb ./cuDNN/
%cd cuDNN
# !cd "/content/gdrive/MyDrive/darknet/cuDNN/"
# !cd "/content/drive/MyDrive/darknet/cuDNN/"

In [ ]:
ls

/content/gdrive/MyDrive/darknet/cuDNN/cudnn-local-repo-ubuntu2204-9.13.0_1.0-1_amd64.deb

In [ ]:
## Instale o instalador local (.deb) que configura o repositório APT:
!sudo dpkg -i cudnn-local-repo-ubuntu2204-9.13.0_1.0-1_amd64.deb

(Este dpkg -i cria um snapshot do repositório em /var/cudnn-local-* e adiciona metadados APT.)

In [ ]:
## Importe a chave GPG do repositório local para o keyring do sistema:
!sudo cp /var/cudnn-local-*/cudnn-*-keyring.gpg /usr/share/keyrings/
# !sudo cp /var/cudnn-local-repo-ubuntu2204-9.13.0/cudnn-local-B46D6A3D-keyring.gpg /usr/share/keyrings/

In [ ]:
## Atualize o cache APT:
!sudo apt-get update

In [ ]:
## Instale o meta-pacote cuDNN para CUDA 12:
# meta-pacote que instala runtime + dev compatíveis com CUDA 12
!sudo apt-get -y install cudnn9-cuda-12

Ou, se preferir instalar pacotes específicos:

In [ ]:
# sudo apt-get -y install libcudnn9-cuda-12 libcudnn9-dev-cuda-12

Verificações rápidas

In [ ]:
## Verificar pacotes instalados:
!dpkg -l | grep cudnn

In [ ]:
## Verificar bibliotecas no diretório CUDA:
%ls -l /usr/local/cuda-12.5/lib64/libcudnn*

In [ ]:
## Teste funcional (compilar e executar amostra):
!sudo apt-get -y install libcudnn9-samples
!cd $HOME/cudnn_samples_v9/mnistCUDNN
!make
!./mnistCUDNN

(Saída bem-sucedida confirma runtime/cuDNN operacional.)

Notas para uso no Colab

Colab não permite fazer download do instalador cuDNN direto sem login; fluxo recomendado:

Baixe o .deb localmente (já feito).

Faça upload do .deb ao Colab (from google.colab import files; files.upload()) ou disponibilize via link privado.

No Colab, execute os mesmos comandos sudo dpkg -i ..., cp /var/cudnn-local-*/cudnn-*-keyring.gpg /usr/share/keyrings/, apt-get update e apt-get install cudnn9-cuda-12.

Atenção: permissões sudo e persistência no Colab podem variar; ajuste conforme saída do notebook.

Riscos / pontos de atenção

Apenas uma versão principal de cuDNN 9 para CUDA 12 deve estar ativa por vez (evite múltiplos libcudnn para versões diferentes).

Confirme versão do driver NVIDIA (nvidia-smi) — driver deve atender requisito mínimo indicado na support matrix do cuDNN.

Guarde o .deb original em darknet/cuDNN para auditoria/rollback.

Referências

NVIDIA — Installing cuDNN on Linux (package manager / local installer).
NVIDIA Docs

NVIDIA — cuDNN Downloads / Release 9.13.0 (página de download / nomes de pacotes).
NVIDIA Developer

Guia prático de instalação CUDA/cuDNN (procedimentos APT / verificação) — exemplos e melhores práticas.
DigitalOcean

In [ ]:
# # Estamos descompactando os arquivos cuDNN da sua pasta do Drive diretamente para as pastas CUDA da VM
# !tar -xzvf gdrive/My\ Drive/darknet/cuDNN/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
# !chmod a+r /usr/local/cuda/include/cudnn.h

# # Agora verificamos a versão que já instalamos. Podemos comentar esta linha em execuções futuras
# !cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

## PASSO 4. Instalando o Darknet


Ótimo!! Temos tudo o que é necessário para começar a trabalhar com o Darknet.

Este notebook funciona com uma versão ligeiramente modificada do Darknet, baseada no [repositório AlexeyAB Darknet](https://github.com/AlexeyAB/darknet/).
As alterações aplicadas ao repositório original são:
* Permitindo o uso de espaços nos arquivos de configuração do Darknet _**obj.data**_. Necessário para trabalhar diretamente com o Google Drive.
* Remoção de alguns logs a cada período. O repositório original grava mais logs do que o notebook Colab consegue sincronizar. Isso cria uma longa fila durante o treinamento. Esta versão só mostra os resultados após cada iteração.

Você pode dar uma olhada no código no [repositório do GitHub](https://github.com/kriyeng/darknet/)

> _**TRUQUE**: Como queremos executar o notebook rapidamente todas as vezes, compilaremos o Darknet apenas na primeira vez que executarmos este notebook. Em seguida, salvaremos a versão compilada no seu disco. Para as execuções futuras, copiaremos a versão compilada em vez de compilar novamente.

**Aqui vai nosso primeiro truque para acelerar o tempo de carregamento em tempo de execução**

* Na primeira vez, iremos:
* Clonar e compilar o projeto Darknet.
* Copiar a versão compilada para o nosso Google Drive Floder

* Nas próximas vezes, em vez de compilá-lo novamente a cada carregamento em tempo de execução, copiaremos a versão compilada para a nossa máquina virtual!

Ao compilar pela primeira vez, a última linha de saída deve ser algo como isto:

`g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv` -DGPU (...)`

## PASSO 4-A. Clonando e compilando o Darkent. SÓ PRECISA SER EXECUTADO NA PRIMEIRA EXECUÇÃO!!


Nesta etapa, clonaremos o repositório do Darkent e o compilaremos.
* Clonar Repositório
* Compilar o Darknet
* Copiar a versão compilada para o Drive

Ao final da compilação, a última linha de saída deve ser algo como isto:

_`g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv` -DGPU (...)_`

In [ ]:
# # Deixe este código sem comentário na primeira execução do seu notebook ou se precisar recompilar o darknet novamente.
# # Comente este código nas execuções futuras.
# !git clone https://github.com/kriyeng/darknet/
# # %cd darknet
# %cd /content/drive/MyDrive/darknet

# # Verifique a pasta
# !ls

# # Tenho uma branch onde fiz as alterações comentadas acima
# !git checkout feature/google-colab

# #Compilar Darknet
# !make

# #Copia a versão compilada do Darknet para o Google Drive
# # !cp ./darknet /content/gdrive/My\ Drive/darknet/bin/darknet
# !cp ./darknet /content/drive/My\ Drive/darknet/bin/darknet

## PASSO 4-B. Copiando a versão compilada do Darknet do Drive. DESCOMENTAR APÓS A PRIMEIRA EXECUÇÃO


Copie a versão compilada do Darknet do Drive para a VM.
* Crie a pasta Darknet local
* Copie o arquivo Darknet
* Defina as permissões de execução

In [ ]:
!ls

In [ ]:
# ## Descomente após a primeira execução, quando tiver uma cópia do Darkent compilado no seu Google Drive

# ## Crie um diretório para o Darknet e mova-o para lá
# !mkdir darknet
# %cd darknet

# ## Copie a versão compilada do Darkent para a unidade local da VM
# !cp /content/gdrive/My\ Drive/darknet/bin/darknet ./darknet

# ## Defina as permissões de execução para o Darknet
# !chmod +x ./darknet

## PASSO 5. Configuração de tempo de execução concluída!


Vamos conferir!
Se você estiver executando este notebook pela primeira vez, pode executar as seguintes células para verificar se tudo está funcionando conforme o esperado!




## Alguns Utilitários

Vamos adicionar alguns utilitários que talvez possam ser úteis.

Esses utilitários são:
* imgShow() - Ajudará a exibir uma imagem na VM remota
* download() - Permitirá que você obtenha algum arquivo do seu notebook caso precise
* upload() - Você pode enviar arquivos para sua pasta atual na VM remota.

Agradecimentos a [Ivan Goncharov](https://twitter.com/Ivangrov) por essas ajudas!

In [ ]:
## Baixar arquivos
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()


def upload():
  from google.colab import files
  uploaded = files.upload()
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
# Célula não necessária
# Obter pesos yolov3
!wget https://pjreddie.com/media/files/yolov3.weights

**NOTA:** O teste a seguir só funcionará quando a darknet for compilada em tempo de execução. Esta demonstração usa alguns dados das pastas originais da darknet. Para seus projetos de detecção de objetos, você terá esses arquivos necessários em sua pasta local.

In [ ]:
# # Célula não necessária
# # Executa darknet usando o modelo YOLOv3 com pesos pré-treinados para detectar objetos em 'person.jpg'
# !./darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg -dont-show

# # Exibe o resultado usando o auxiliar imgShow()
# imShow('predictions.jpg')

## Se você consegue ver a mesma imagem abaixo, parabéns!! Neste ponto, você já configurou e está usando o Darknet!

# PARTE 2. Treinando o YOLO



> _**DICA:** Sempre que quiser executar todas as suas células automaticamente, você pode ir em **> Menu > Tempo de Execução > Executar Tudo**. Talvez você não queira executar o notebook inteiro. Você pode escrever a seguinte célula onde deseja interromper o processo e descomentar o comando **`assert False`**. Isso gerará um erro e interromperá a execução de mais células. Agradecimentos a: [Este tópico](https://groups.google.com/forum/#!topic/jupyter/ELftSFSiedQ)_




In [ ]:
# não vá além daqui com Executar Tudo
#assert Falso

## PARTE 2. PASSO 0. Preparando seus dados e arquivos de configuração



Antes de prosseguir, vamos dar uma olhada nos arquivos de configuração que você precisa ter em sua unidade local _`darknet`_

Você pode baixar a folha de dicas [aqui](http://blog.ibanyez.info/download/B20190410T000000072.png)

Se precisar de explicações mais detalhadas sobre como preparar seus conjuntos de dados, anotações e aprendizado profundo, visite [Como treinar o YOLOv3 usando o framework Darknet e otimizar os tempos de carregamento da VM](link da postagem)

> **DICA:** Você precisa configurar cuidadosamente os caminhos em seus arquivos de configuração. O arquivo _*obj.data*_ precisa ter espaços no caminho, com escape de _**\**_. Assim: **_/content/gdrive/My\ Drive/darknet/train.txt_**. Mas, nos arquivos **_train.txt_** e **_test.txt_**, isso não acontece!




## PARTE 2. ETAPA 1. Carregando arquivos para a unidade local da VM


A velocidade da rede entre o Google Drive e a VM Colab pode afetar a velocidade de acesso às imagens do seu conjunto de dados durante o treinamento.

Você pode ter três abordagens possíveis, dependendo do tamanho do seu conjunto de dados.

> _**NOTA:** Esta etapa não é necessária para todos os arquivos de configuração e pesos. Eles podem ser acessados ​​diretamente do Google Drive sem problemas consideráveis ​​de desempenho. **Esta etapa é apenas para as imagens e anotações do conjunto de dados**_

* **Opção 1** - Você pode tentar usar diretamente os arquivos da pasta _`img/`_ do Google Drive. Dependendo do seu conjunto de dados, talvez isso seja mais do que suficiente.
* **Opção 2** - Antes de iniciar o treinamento, copie seu conjunto de dados do Google Drive para o sistema de arquivos da VM local. Talvez seja uma boa prática copiar como um único arquivo tar e descompactar na pasta local _`img/`_ da sua VM.
* **Opção 3** - Se o seu conjunto de dados for muito grande, talvez você possa fazer o upload para um repositório Git e clonar a partir dele. Normalmente, o tempo de transferência entre eles é muito melhor. Se você tiver que decidir, tenho a impressão de que o Bitbucket tem tempos de transferência mais rápidos do que o GitHub, mas, por favor, não tome isso como uma confirmação. **Não fiz testes específicos sobre isso, posso estar errado!**




#### PARTE 2. PASSO 1 - Opção 1. Usando arquivos diretamente do Google Drive.


Você não precisa fazer nada aqui. Seu **_train.txt_** deve ter o caminho correto:
* **/content/grdive/My Drive/darknet/img/image001.jpg**. Como mencionado anteriormente, não use espaços em branco de escape para os caminhos em _**train.txt**_ e _**test.txt**_

#### PARTE 2. PASSO 1 - Opção 2A. Copiando arquivos do Google Drive para o sistema de arquivos local da VM.


Execute a célula a seguir para copiar seus arquivos

In [ ]:
## Copie arquivos do Google Drive para o sistema de arquivos local da VM
!cp -r "/content/gdrive/My Drive/darknet/img" ./img


#### PARTE 2. ETAPA 1 - Opção 2B. Copiar os arquivos compactados do Google Drive para o sistema de arquivos local da VM e descompactar localmente.


Execute a célula a seguir para copiar seus arquivos e descompactá-los.
Você pode usar o comando _*!ls*_ para verificar qual é o caminho correto que você precisa configurar no seu _*train.txt*_ para acessar corretamente as imagens do seu conjunto de dados.

In [ ]:
# Copie seu arquivo compactado
#!cp -r "/content/gdrive/Meu Drive/darknet/img/img.tgz" ./img

# Descompacte o arquivo compactado
#!tar -xzvf ./img/img.tgz

#### PARTE 2. PASSO 1 - Opção 3. Clone seu conjunto de dados de imagens de um repositório Git. Parece ser a opção mais rápida.


Execute a célula a seguir para clonar seu repositório de conjuntos de dados para o sistema de arquivos local da VM.

In [ ]:
# Git clone directly to ./img folder
#!git clone https://[your-repository] ./img

# Check the result - Uncomment when you checked for speed up further runs
#!ls -la ./img

## PARTE 3. Finalmente, treine seu modelo!



Ao executar o comando a seguir, seu modelo iniciará o treinamento.

Você terá uma linha de log por época. A cada iteração, você verá como está o seu treinamento.

> **DICA: O Darknet copia um backup dos seus pesos treinados a cada 100 iterações. Como mágica, este arquivo será sincronizado com o seu disco local no seu computador, assim como os backups que o Darknet faz a cada 1000 iterações, salvando-o em um arquivo separado.**





In [ ]:
!./darknet detector train "/content/gdrive/My Drive/darknet/obj.data" "/content/gdrive/My Drive/darknet/yolov3.cfg" "/content/gdrive/My Drive/darknet/darknet53.conv.74" -dont_show

## DICAS E TRUQUES DE DESEMPENHO



* **Acelere o tempo de carregamento do tempo de execução:** Depois de verificar se tudo está funcionando, você pode remover células ou comentar linhas de código desnecessárias para reduzir o tempo de carregamento a cada execução.

* **Como manter seu notebook ativo por mais tempo?:** Mantenha o navegador com o notebook aberto. Se você fechar o navegador, o notebook atingirá o tempo de inatividade e será removido do Serviço de Nuvem Colab. (90 minutos)

* **Reexecute seu treinamento após atingir o tempo limite para tempos de execução do Colab (12 horas):**
* Abra um novo notebook ou reconecte o atual.
* Comente a célula acima e descomente a célula abaixo.
* No seu computador local, copie o arquivo **backup/yolov3_last.weights** para a pasta **weights/** do seu computador local.
* Execute Executar tudo no menu **> > Tempo de Execução > Executar Tudo**
* _A etapa de cópia não é absolutamente necessária, mas gosto de manter uma cópia da última sessão de treinamento e não sobrescrever este arquivo nos próximos treinamentos._

In [ ]:
# Inicie o treinamento no ponto em que o último tempo de execução terminou
#!./darknet detector train "/content/gdrive/My Drive/darknet/obj.data" "/content/gdrive/My Drive/darknet/yolov3.cfg" "/content/gdrive/My Drive/darknet/weights/yolov_last.weights" -dont_show

## SOLUÇÃO DE PROBLEMAS


Os principais problemas que você pode enfrentar se o seu modelo apresentar um erro são:

* Imagens ou arquivos não encontrados. Verifique a imagem da **folha de dicas do Yolov3** acima e verifique se está tudo ok.
* Se você configurou incorretamente seus **filtros** e **classes** no **yolov3.cfg**, consulte a **folha de dicas do Yolov3** acima.
* Você pode enfrentar alguns erros de falta de memória ou de biblioteca, principalmente devido à falta de alguns pré-requisitos. Nesse caso, verifique as versões das bibliotecas atuais instaladas na sua VM Colab. Você pode encontrar mais informações nos primeiros passos deste caderno.
* Os parâmetros **Lote** e **subdivisões** no seu **yolov3.cfg** também podem afetar a alocação de memória. Consulte o repositório original [Aqui]() para mais detalhes.



### ATUALIZAÇÃO DA SOLUÇÃO DE PROBLEMAS


Tenha cuidado ao preparar seus arquivos no Windows. Se você usar **CRLF** em seus arquivos em vez de **LF**, poderá ter problemas para abrir os arquivos corretamente. - Obrigado a [Satya Shetty](https://twitter.com/satyashetty) por compartilhar esse problema!

## Sobre mim



Você pode encontrar a postagem original com mais explicações sobre este notebook em [Como treinar YOLOv3 usando o framework Darknet e otimizar os tempos de carregamento da VM] (link da postagem)

Sou David Ibañez, de Barcelona. Sinta-se à vontade para entrar em contato!

* Você pode visitar meu blog em [Dev-ibanyez.info](http://blog.ibanyex.info)
* Você pode entrar em contato comigo no [Twitter](https://twitter.com/dav_ibanez)
* Você pode entrar em contato ou contribuir para este notebook no [Github](https://github.com/kriyeng/yolo-on-colab-notebook/)
* Você pode comentar na postagem [dev.to sobre este notebook] (PENDENTE)

Obrigado por ler este notebook! :clap: :clap: :clap:

## FONTES

#### Outras fontes






* Site original do YOLO [Página de Joseph Redmon](https://pjreddie.com/darknet/yolo/)
* Repositório darknet de AlexeyAB [github](https://github.com/AlexeyAB/darknet/)
* O [caderno](https://github.com/ivangrov/YOLOv3-GoogleColab/blob/master/YOLOv3_GoogleColab.ipynb) de Ivan Goncharov me inspirou a experimentar o Google Colab e acabei criando este caderno.